This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [21]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [22]:
# select model to train

model_inventory = {'mktprob': {'X_columns': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSig},
                   'mktprob_soft': {'X_columns': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSoft},
                    'mktprob_MLR': {'X_columns': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': MLR},
                   'AlunOwen_v0': {'X_columns': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v1': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v2': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinDropReluLinSoft},
                   'AlunOwen_v3': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR}
                               }

model = 'AlunOwen_v3'
X_columns = model_inventory[model]['X_columns']
learning_rate = model_inventory[model]['learning_rate']
epochs = model_inventory[model]['epochs']
vacant_stall_indicator = model_inventory[model]['vacant_stall_indicator']
bias = model_inventory[model]['bias']
model_architecture = model_inventory[model]['model_architecture']

In [23]:
# read in data

y_columns = ["win"]

train_data_fn = "data\\runners_train.csv"
test_data_fn = "data\\runners_test.csv"

train_data = RacesDataset(train_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator)
test_data = RacesDataset(test_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, scalar=train_data.scalar)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [24]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.iloc[:, train_data.X_columns].head()

age                                                              \
stall_number   1    2    3    4    5    6    7    8    9    10   11   12   13   
race_id                                                                         
11504         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0  0.0  0.0   
11505         4.0  6.0  9.0  7.0  5.0  8.0  5.0  5.0  5.0  4.0  5.0  0.0  0.0   
11506         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0   
11507         5.0  4.0  4.0  4.0  4.0  4.0  8.0  4.0  4.0  0.0  0.0  0.0  0.0   
11508         7.0  4.0  5.0  4.0  5.0  4.0  5.0  5.0  4.0  0.0  0.0  0.0  0.0   

                            blinkers                                          \
stall_number   14   15   16       1    2    3    4    5    6    7    8    9    
race_id                                                                        
11504         0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                cheekpieces                 \
stall_number   10   11   12   13   14   15   16          1    2    3    4    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             daysLTO                                                          \
stall_number      1      2      3      4      5      6      7      8      9    
race_id                                                                        
11504          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0    0.0   
11505          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11506          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11507          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11508          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   

                                                    entire                 \
stall_number     10     11   12   13   14   15   16     1    2    3    4    
race_id                                                                     
11504           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11505         365.0  365.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11506         365.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11507           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11508           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
1150

In [25]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]

# torch.manual_seed(0)
net = model_architecture(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in net.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: AlunOwen_v3
Layer: neural_network.0.weights | Size: torch.Size([20]) | Values : Parameter containing:
tensor([-0.3507, -0.3330,  0.1525,  0.3119,  0.2764, -0.0767,  0.4799,  0.2086,
         0.1461,  0.4840, -0.1382, -0.0034, -0.1200,  0.0239,  0.0616, -0.1162,
        -0.0818,  0.0359, -0.1207, -0.0234], requires_grad=True) 



In [26]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = net(X)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([6])
torch.Size([1, 320])


In [27]:
%env WANDB_NOTEBOOK_NAME 'C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "device": device,
    "model": model,
    "X_columns": X_columns,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "vacant_stall_indicator": vacant_stall_indicator,
    "bias": bias,
    "model_architecture": list(net.modules())
    }
)

env: WANDB_NOTEBOOK_NAME='C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'


In [28]:
# optimizing model parameters

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer, device)
    (acc, loss) = test_loop(test_dataloader, net, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

Epoch 1
-------------------------------
loss: 2.772957  [   64/16620]
loss: 2.760281  [ 6464/16620]
loss: 2.811059  [12864/16620]
Test Error: 
 Accuracy: 15.9%, Avg loss: 2.753920 

Epoch 2
-------------------------------
loss: 2.769868  [   64/16620]
loss: 2.747210  [ 6464/16620]
loss: 2.799972  [12864/16620]
Test Error: 
 Accuracy: 18.0%, Avg loss: 2.741208 

Epoch 3
-------------------------------
loss: 2.764286  [   64/16620]
loss: 2.733308  [ 6464/16620]
loss: 2.787822  [12864/16620]
Test Error: 
 Accuracy: 19.1%, Avg loss: 2.728886 

Epoch 4
-------------------------------
loss: 2.756614  [   64/16620]
loss: 2.720457  [ 6464/16620]
loss: 2.775795  [12864/16620]
Test Error: 
 Accuracy: 19.9%, Avg loss: 2.717927 

Epoch 5
-------------------------------
loss: 2.748583  [   64/16620]
loss: 2.709217  [ 6464/16620]
loss: 2.764917  [12864/16620]
Test Error: 
 Accuracy: 20.4%, Avg loss: 2.708689 

Epoch 6
-------------------------------
loss: 2.741418  [   64/16620]
loss: 2.699215  [ 64

In [29]:
# finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▄▆▇▆▇▇▇▇▇▇▇██████▇██▇██████████████████
loss,█▆▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.22897
loss,2.65105


In [30]:
for para_name, para_vals in net.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")